In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data269


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 16.4MB/s eta 0:00:01


In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [5]:
import numpy as np
import paddle
print(paddle.__version__)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd

2.1.0


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [6]:
colnames = ['房屋面积']+['房价']
print_data = pd.read_csv('./房价预测/data/data.txt',names = colnames)
print_data.head()

,房屋面积,房价
0,98.87,599.0
1,68.74,450.0
2,89.24,440.0
3,129.19,780.0
4,61.64,450.0


In [9]:
global x_raw,train_data,test_data
data = np.loadtxt('./房价预测/data/data.txt',delimiter=',')
x_raw = data.T[0].copy() 

#axis=0,表示按列计算
#data.shape[0]表示data中一共有多少列
maximums,minimums,avgs = data.max(axis=0),data.min(axis=0),data.sum(axis=0)/data.shape[0]
print("the raw area :",data[:,0].max(axis = 0))
#归一化
data[:,0] = (data[:,0]-avgs[0])/(maximums[0]-minimums[0])
print('normalization:',data[:,0].max(axis = 0))
plt.plot(data[:,0],data[:,1])
plt.show()

the raw area : 199.96
normalization: 0.6587568635148239


In [11]:
#划分数据集
ratio = 0.8
offset = int(data.shape[0]*ratio)
train_data = data[0:offset]
test_data = data[offset:]
print('数据总个数：',len(data))
print('训练集数据个数:',len(train_data))
print('测试集数据个数:',len(test_data))

数据总个数： 870
训练集数据个数: 696
测试集数据个数: 174


In [12]:
import paddle
from paddle.io import Dataset

class MyDataset(Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self, Data):
        """
        步骤二：实现构造函数，定义数据集大小
        """
        super(MyDataset, self).__init__()
        self.num_samples = len(Data)
        self.Data = Data.astype('float64')

    def __getitem__(self, index):
        """
        步骤三：实现__getitem__方法，定义指定index时如何获取数据，并返回单条数据（训练数据，对应的标签）
        """
        data = self.Data[index,0]
        label = self.Data[index,1]

        return data, label

    def __len__(self):
        """
        步骤四：实现__len__方法，返回数据集总数目
        """
        return self.num_samples




In [13]:
BATCH_SIZE = 1
train_loader = paddle.io.DataLoader(MyDataset(train_data), batch_size=BATCH_SIZE, shuffle=True)
test_loader = paddle.io.DataLoader(MyDataset(test_data), batch_size=BATCH_SIZE, shuffle=True)

In [14]:
#建立模型
class net(paddle.nn.Layer):
    def __init__(self):
        super(net, self).__init__()
        self.fc = paddle.nn.Sequential(
        paddle.nn.Linear(1, 1),
        paddle.nn.ReLU(),
        paddle.nn.Linear(1, 1)
        )

    def forward(self, inputs):
        pred = self.fc(inputs)
        return pred


In [15]:
paddle.set_default_dtype("float64")
# step3:训练模型
model = paddle.Model(net())
model.prepare(paddle.optimizer.Adam(parameters=model.parameters()),
              paddle.nn.MSELoss())
model.fit(train_loader, test_loader, epochs=5, batch_size=20, verbose=1)


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 110/696 [===>..........................] - loss: 159973.4326 - ETA: 1s - 2ms/ste

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 210/696 [========>.....................] - loss: 336170.3540 - ETA: 0s - 2ms/stepstep 696/696 [==============================] - loss: 210097.7786 - 2ms/step         
Eval begin...
step 174/174 [==============================] - loss: 448046.4839 - 876us/step         
Eval samples: 174
Epoch 2/5
step 696/696 [==============================] - loss: 2555944.5906 - 2ms/step        
Eval begin...
step 174/174 [==============================] - loss: 120222.4829 - 960us/step        
Eval samples: 174
Epoch 3/5
step 696/696 [==============================] - loss: 842910.1075 - 2ms/step        
Eval begin...
step 174/174 [==============================] - loss: 719274.0883 - 995us/step         
Eval samples: 174
Epoch 4/5
step 696/696 [==============================] - loss: 187031.5694 - 2ms/step         
Eval begin...
step 174/174 [==============================] - loss: 120735.8642 - 915us/step        
Eval samples: 174
Epoch 5/5
step 696/696 [==============================] - loss:

## Numpy实现

In [28]:
# 导入需要用到的package
import numpy as np
import matplotlib.pyplot as plt


def load_data():
    # 从文件导入数据
    '''
    datafile = './housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])
'''
    data = np.loadtxt('./房价预测/data/data.txt',delimiter=',')
    x_raw = data.T[0].copy() 

    #axis=0,表示按列计算
    #data.shape[0]表示data中一共有多少列
    maximums,minimums,avgs = data.max(axis=0),data.min(axis=0),data.sum(axis=0)/data.shape[0]
    print("the raw area :",data[:,0].max(axis = 0))
    #归一化
    data[:,0] = (data[:,0]-avgs[0])/(maximums[0]-minimums[0])
    print('normalization:',data[:,0].max(axis = 0))

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data


class Network(object):
    def __init__(self, num_of_weights):
        # 随机产生w的初始值
        # 为了保持程序每次运行结果的一致性，此处设置固定的随机数种子
        # np.random.seed(0)
        self.w = np.random.randn(num_of_weights, 1)
        self.b = 0.

    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        return z

    def loss(self, z, y):
        error = z - y
        num_samples = error.shape[0]
        cost = error * error
        cost = np.sum(cost) / num_samples
        return cost

    def gradient(self, x, y):
        z = self.forward(x)
        N = x.shape[0]
        gradient_w = 1. / N * np.sum((z - y) * x, axis=0)
        gradient_w = gradient_w[:, np.newaxis]
        gradient_b = 1. / N * np.sum(z - y)
        return gradient_w, gradient_b

    def update(self, gradient_w, gradient_b, eta=0.01):
        self.w = self.w - eta * gradient_w
        self.b = self.b - eta * gradient_b

    def train(self, training_data, num_epoches, batch_size=10, eta=0.01):
        n = len(training_data)
        losses = []
        for epoch_id in range(num_epoches):
            # 在每轮迭代开始之前，将训练数据的顺序随机的打乱，
            # 然后再按每次取batch_size条数据的方式取出
            np.random.shuffle(training_data)
            # 将训练数据进行拆分，每个mini_batch包含batch_size条的数据
            mini_batches = [training_data[k:k + batch_size] for k in range(0, n, batch_size)]
            for iter_id, mini_batch in enumerate(mini_batches):
                # print(self.w.shape)
                # print(self.b)
                x = mini_batch[:, :-1]
                y = mini_batch[:, -1:]
                a = self.forward(x)
                loss = self.loss(a, y)
                gradient_w, gradient_b = self.gradient(x, y)
                self.update(gradient_w, gradient_b, eta)
                losses.append(loss)
                print('Epoch {:3d} / iter {:3d}, loss = {:.4f}'.format(epoch_id, iter_id, loss))

        return losses


# 获取数据
train_data, test_data = load_data()

# 创建网络
net = Network(1)
# 启动训练
losses = net.train(train_data, num_epoches=50, batch_size=100, eta=0.1)

# 画出损失函数的变化趋势
plot_x = np.arange(len(losses))
plot_y = np.array(losses)
plt.plot(plot_x, plot_y)
plt.show()

the raw area : 199.96
normalization: 0.6587568635148239
Epoch   0 / iter   0, loss = 531435.8569
Epoch   0 / iter   1, loss = 383205.2484
Epoch   0 / iter   2, loss = 335050.7708
Epoch   0 / iter   3, loss = 315788.1438
Epoch   0 / iter   4, loss = 182908.0060
Epoch   0 / iter   5, loss = 157463.7102
Epoch   0 / iter   6, loss = 139679.6585
Epoch   1 / iter   0, loss = 144945.9604
Epoch   1 / iter   1, loss = 92951.6929
Epoch   1 / iter   2, loss = 161047.5038
Epoch   1 / iter   3, loss = 123659.3931
Epoch   1 / iter   4, loss = 102778.7896
Epoch   1 / iter   5, loss = 148764.1256
Epoch   1 / iter   6, loss = 93541.3783
Epoch   2 / iter   0, loss = 108199.1016
Epoch   2 / iter   1, loss = 61548.1197
Epoch   2 / iter   2, loss = 62692.4809
Epoch   2 / iter   3, loss = 79263.5686
Epoch   2 / iter   4, loss = 89952.5938
Epoch   2 / iter   5, loss = 80324.6455
Epoch   2 / iter   6, loss = 98261.2465
Epoch   3 / iter   0, loss = 52558.3018
Epoch   3 / iter   1, loss = 73162.7807
Epoch   3 /